### Importação das Bibliotecas
##### os / time → manipulação de diretórios e utilidades gerais

##### numpy → operações matemáticas e vetoriais

##### cv2 (OpenCV) → leitura e processamento de imagens (Canny, FFT, HaarCascade etc.)

##### joblib → salvar e carregar modelos (Logistic Regression, scaler, stacking)

##### sklearn → modelos tradicionais e métricas

##### tensorflow / keras → criação, treino e avaliação da CNN

In [ ]:
import os
import time
import numpy as np
import cv2
import joblib

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    classification_report,
)

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

### Configurações do Projeto

##### Define caminhos para treino e validação.

##### Ajusta parâmetros do modelo: tamanho da imagem, batch, número de épocas.

##### Define extensões aceitas (incluindo WebP).

##### Garante existência do diretório models/.

##### Define os arquivos onde serão salvos os modelos treinados.

In [ ]:
BASE_DIR = "rvf10k/rvf10k"
TRAIN_DIR = os.path.join(BASE_DIR, "train")
VALID_DIR = os.path.join(BASE_DIR, "valid")

IMG_SIZE = (128, 128)
BATCH_SIZE = 32
EPOCHS = 15
RANDOM_SEED = 42

# formatos aceitos (inclui webp)
IMG_EXTS = (".jpg", ".jpeg", ".png", ".webp")

# diretórios de saída
os.makedirs("models", exist_ok=True)
CNN_MODEL_PATH = "models/cnn_rvf10k_v2.keras"
DET_MODEL_PATH = "models/det_logreg_v2.joblib"
STACK_MODEL_PATH = "models/stacking_v2.joblib"
DET_SCALER_PATH = "models/det_scaler_v2.joblib"

##### Carrega automaticamente imagens organizadas em pastas por classe.

##### Converte para batches já redimensionados em 128×128.

##### Otimiza carregamento com cache() e prefetch().

In [ ]:
def load_image_datasets():
    train_ds = keras.utils.image_dataset_from_directory(
        TRAIN_DIR,
        labels="inferred",
        label_mode="binary",
        batch_size=BATCH_SIZE,
        image_size=IMG_SIZE,
        shuffle=True,
        seed=RANDOM_SEED,
    )

    val_ds = keras.utils.image_dataset_from_directory(
        VALID_DIR,
        labels="inferred",
        label_mode="binary",
        batch_size=BATCH_SIZE,
        image_size=IMG_SIZE,
        shuffle=False,
    )

    AUTOTUNE = tf.data.AUTOTUNE
    train_ds = train_ds.cache().shuffle(2000).prefetch(AUTOTUNE)
    val_ds = val_ds.cache().prefetch(AUTOTUNE)

    return train_ds, val_ds

### Construção da CNN
##### Normaliza a imagem para valores entre 0 e 1.

### Bloco Convolucional
##### Convolução (extração de padrões)

##### BatchNorm (estabiliza treinamento)

##### MaxPooling (reduz dimensionalidade)

##### Os filtros aumentam a complexidade progressivamente.

### Camadas finais da CNN
##### GAP → reduz drasticamente parâmetros.

##### Dropout → reduz overfitting.

##### Dense(64) → representação aprendida.

##### Sigmoid → saída entre 0 e 1 (probabilidade fake).

In [ ]:
def build_cnn_model(input_shape=(128, 128, 3)):
    inputs = keras.Input(shape=input_shape)
    x = layers.Rescaling(1.0 / 255)(inputs)

    def block(x, filters):
        x = layers.Conv2D(filters, 3, padding="same", activation="relu")(x)
        x = layers.BatchNormalization()(x)
        x = layers.MaxPooling2D()(x)
        return x

    x = block(x, 32)
    x = block(x, 64)
    x = block(x, 128)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.6)(x)
    x = layers.Dense(64, activation="relu")(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)

    model = keras.Model(inputs, outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(1e-3),
        loss="binary_crossentropy",
        metrics=[
            "accuracy",
            keras.metrics.Precision(name="precision"),
            keras.metrics.Recall(name="recall"),
        ],
    )
    return model

### Extração de Features Determinísticas
##### Usar o HaarCascade para detectar rosto antes da análise

In [ ]:
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

def extract_face(gray):
    faces = face_cascade.detectMultiScale(gray, 1.1, 5, minSize=(30, 30))
    if len(faces) == 0:
        return cv2.resize(gray, IMG_SIZE)
    x, y, w, h = max(faces, key=lambda f: f[2] * f[3])
    return cv2.resize(gray[y : y + h, x : x + w], IMG_SIZE)

### Local Binary Patterns
##### Gera um histograma de textura com 256 valores, sendo muito útil para distinguir pele real de pele sintética.

In [ ]:
def lbp_features(img):
    img = img.astype(np.uint8)
    h, w = img.shape
    if h < 3 or w < 3:
        img = cv2.resize(img, (max(3, w), max(3, h)))
        h, w = img.shape

    lbp = np.zeros((h - 2, w - 2), dtype=np.uint8)

    for i in range(1, h - 1):
        for j in range(1, w - 1):
            center = img[i, j]
            code = 0
            code |= (img[i - 1, j - 1] > center) << 7
            code |= (img[i - 1, j] > center) << 6
            code |= (img[i - 1, j + 1] > center) << 5
            code |= (img[i, j + 1] > center) << 4
            code |= (img[i + 1, j + 1] > center) << 3
            code |= (img[i + 1, j] > center) << 2
            code |= (img[i + 1, j - 1] > center) << 1
            code |= (img[i, j - 1] > center) << 0
            lbp[i - 1, j - 1] = code

    hist, _ = np.histogram(lbp.ravel(), bins=256, range=(0, 256), density=True)
    return hist

### Extração completa de features

##### média / desvio padrão →	intensidade geral da imagem
##### Laplacian variance → mede nitidez / blur
##### edge density →	quantidade de bordas presentes
##### FFT high frequency ratio → mede ruído e padrões artificiais
##### LBP histogram →	textura local detalhada

In [ ]:
def deterministic_features_from_image(path):
    img = cv2.imread(path)

    if img is None:
        raise ValueError(
            f"Falha ao carregar {path} — formato pode não ser suportado pelo OpenCV!"
        )

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    face = extract_face(gray)

    mean_intensity = np.mean(face)
    std_intensity = np.std(face)
    lap_var = cv2.Laplacian(face, cv2.CV_64F).var()
    edges = cv2.Canny(face, 100, 200)
    edge_density = np.mean(edges > 0)

    f = np.fft.fft2(face)
    fshift = np.fft.fftshift(f)
    mag = np.abs(fshift)
    h, w = mag.shape
    cy, cx = h // 2, w // 2
    low = mag[cy - h // 4 : cy + h // 4, cx - w // 4 : cx + w // 4].sum()
    total = mag.sum() + 1e-8
    hf_ratio = (total - low) / total

    hist = lbp_features(face)

    return np.concatenate(
        [
            np.array(
                [mean_intensity, std_intensity, lap_var, edge_density, hf_ratio],
                dtype=np.float32,
            ),
            hist.astype(np.float32),
        ]
    )

### Carregar dataset determinístico
##### Percorre pastas /real e /fake, extrai features para cada imagem e retorna vetores X (features) e y (labels)

In [ ]:
def load_det_dataset(base_dir):
    X, y = [], []

    for label, cls_name in [(0, "real"), (1, "fake")]:
        cls_dir = os.path.join(base_dir, cls_name)

        if not os.path.exists(cls_dir):
            print(f"[ERRO] Diretório não encontrado: {cls_dir}")
            continue

        files = [f for f in os.listdir(cls_dir) if f.lower().endswith(IMG_EXTS)]

        print(f"[INFO] {cls_name}: {len(files)} imagens encontradas")

        for fname in files:
            path = os.path.join(cls_dir, fname)
            try:
                features = deterministic_features_from_image(path)
                X.append(features)
                y.append(label)
            except Exception as e:
                print(f"[WARN] Falhou: {path} → {e}")

    return np.array(X), np.array(y)

### Função de Avaliação
##### Calcula Acurácia, Precisão, Recall, F1, Classification Report

In [ ]:
def evaluate(name, y_true, y_prob):
    y_pred = (y_prob >= 0.5).astype(int)
    acc = accuracy_score(y_true, y_pred)
    prec, rec, f1, _ = precision_recall_fscore_support(
        y_true, y_pred, average="binary", zero_division=0
    )

    print(f"\n==== {name} ====")
    print(f"Acurácia : {acc:.4f}")
    print(f"Precisão : {prec:.4f}")
    print(f"Recall   : {rec:.4f}")
    print(f"F1       : {f1:.4f}")
    print(classification_report(y_true, y_pred))

### Função Principal – main()
##### Etapa 1 — Carregar datasets
##### Etapa 2 — Inicializar CNN
##### Etapa 3 — Treinar CNN
##### Etapa 4 — Avaliar CNN
##### Etapa 5 — Treinar modelo determinístico
##### Etapa 6 — STACKING


In [ ]:
def main():

    print("\n[1] Carregando datasets CNN...")
    train_ds, val_ds = load_image_datasets()

    print("\n[2] Construindo CNN...")
    cnn = build_cnn_model()
    cnn.summary()

    callbacks = [
        keras.callbacks.EarlyStopping(
            patience=4, monitor="val_loss", restore_best_weights=True
        )
    ]

    print("\n[3] Treinando CNN...")
    cnn.fit(train_ds, validation_data=val_ds, epochs=EPOCHS, callbacks=callbacks)
    cnn.save(CNN_MODEL_PATH)
    print(f"[OK] CNN salva em {CNN_MODEL_PATH}")

    print("\n[4] Avaliando CNN...")
    y_true_cnn, y_prob_cnn = [], []
    for Xb, yb in val_ds:
        p = cnn.predict(Xb, verbose=0).flatten()
        y_prob_cnn.extend(p)
        y_true_cnn.extend(yb.numpy().flatten())

    y_true_cnn = np.array(y_true_cnn).astype(int)
    y_prob_cnn = np.array(y_prob_cnn)

    evaluate("CNN v2", y_true_cnn, y_prob_cnn)

    print("\n[5] Carregando dataset determinístico...")
    X_det_train, y_det_train = load_det_dataset(TRAIN_DIR)
    X_det_val, y_det_val = load_det_dataset(VALID_DIR)

    print(f"[INFO] X_det_train: {X_det_train.shape}")
    print(f"[INFO] X_det_val  : {X_det_val.shape}")

    if len(X_det_train) == 0:
        raise RuntimeError(
            "❌ ERRO FATAL: Nenhuma imagem determinística carregada. Verifique extensões e leitura de arquivos."
        )

    scaler = StandardScaler()
    X_det_train_s = scaler.fit_transform(X_det_train)
    X_det_val_s = scaler.transform(X_det_val)

    det_model = LogisticRegression(max_iter=2000)
    det_model.fit(X_det_train_s, y_det_train)

    y_prob_det = det_model.predict_proba(X_det_val_s)[:, 1]
    evaluate("Determinístico v2", y_det_val, y_prob_det)

    joblib.dump(det_model, DET_MODEL_PATH)
    joblib.dump(scaler, DET_SCALER_PATH)

    print("\n[6] Preparando STACKING (CNN + Determinístico)...")

    def list_paths(base):
        paths = []
        labels = []
        for label, cls in [(0, "real"), (1, "fake")]:
            d = os.path.join(base, cls)
            for f in sorted(os.listdir(d)):
                if f.lower().endswith(IMG_EXTS):
                    paths.append(os.path.join(d, f))
                    labels.append(label)
        return np.array(paths), np.array(labels)

    val_paths, val_labels = list_paths(VALID_DIR)

    # CNN alinhada a paths
    y_prob_cnn_aligned = []
    for p in val_paths:
        img = cv2.imread(p)
        if img is None:
            print(f"[WARN] Falha ao reabrir {p} para CNN alinhada, pulando.")
            continue
        rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        rgb = cv2.resize(rgb, IMG_SIZE)
        rgb = np.expand_dims(rgb, 0) / 255.0
        y_prob_cnn_aligned.append(cnn.predict(rgb, verbose=0)[0][0])

    # determinístico alinhado
    X_det_val_aligned = []
    for p in val_paths:
        try:
            feats = deterministic_features_from_image(p)
            X_det_val_aligned.append(feats)
        except Exception as e:
            print(f"[WARN] Falha ao extrair features alinhadas de {p}: {e}")

    X_det_val_aligned = np.array(X_det_val_aligned)
    X_det_val_aligned_s = scaler.transform(X_det_val_aligned)
    y_prob_det_aligned = det_model.predict_proba(X_det_val_aligned_s)[:, 1]

    # Ajuste de tamanho caso alguma imagem tenha sido pulada
    min_len = min(len(y_prob_cnn_aligned), len(y_prob_det_aligned), len(val_labels))
    y_prob_cnn_aligned = np.array(y_prob_cnn_aligned[:min_len])
    y_prob_det_aligned = np.array(y_prob_det_aligned[:min_len])
    y_stack_labels = val_labels[:min_len]

    # STACKING
    X_stack = np.vstack([y_prob_cnn_aligned, y_prob_det_aligned]).T
    stack = LogisticRegression()
    stack.fit(X_stack, y_stack_labels)

    y_prob_stack = stack.predict_proba(X_stack)[:, 1]
    evaluate("Híbrido STACK v2", y_stack_labels, y_prob_stack)

    joblib.dump(stack, STACK_MODEL_PATH)
    print(f"[OK] STACK salvo em {STACK_MODEL_PATH}")

    print("\nTreinamento concluído!")


if __name__ == "__main__":
    main()
